In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Set the weights file 
local_weights_file = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

# Initialize the base model
# Set the input shape and remove the dense layers
pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

# Load the pre-trained weights
pre_trained_model.load_weights(local_weights_file)

# Freeze the weights of the layers.
for layer in pre_trained_model.layers:
  layer.trainable = False

In [3]:
# Choose `mixed7` as the last layer of your base model
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 768)


In [4]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense(31, activation='softmax')(x)     

# Append the dense network to the base model
model = Model(pre_trained_model.input, x) 


In [5]:
# Set the training parameters
model.compile(loss='categorical_crossentropy', optimizer = RMSprop(learning_rate=0.001), metrics=['accuracy'])

In [6]:
# Prepare the ImageDataGenerator
TRAINING_DIR = "TrainModel"
training_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [7]:
train_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(150, 150),
    class_mode='categorical',
    batch_size=64  # Changed batch size to fit your dataset
)

Found 49 images belonging to 31 classes.


In [8]:
# Train the model and evaluate the results
print(f"Number of training samples: {train_generator.samples}")
print(f"Training batch size: {train_generator.batch_size}")

Number of training samples: 49
Training batch size: 64


In [9]:
history = model.fit(
    train_generator,
    epochs=100,
    verbose=2
)

Epoch 1/100


1/1 - 4s - loss: 3.5983 - accuracy: 0.0000e+00 - 4s/epoch - 4s/step
Epoch 2/100
1/1 - 1s - loss: 12.6178 - accuracy: 0.1224 - 780ms/epoch - 780ms/step
Epoch 3/100
1/1 - 1s - loss: 22.9445 - accuracy: 0.1429 - 744ms/epoch - 744ms/step
Epoch 4/100
1/1 - 1s - loss: 18.3117 - accuracy: 0.1020 - 764ms/epoch - 764ms/step
Epoch 5/100
1/1 - 1s - loss: 14.5848 - accuracy: 0.2041 - 774ms/epoch - 774ms/step
Epoch 6/100
1/1 - 1s - loss: 11.3265 - accuracy: 0.1429 - 749ms/epoch - 749ms/step
Epoch 7/100
1/1 - 1s - loss: 8.1335 - accuracy: 0.2041 - 760ms/epoch - 760ms/step
Epoch 8/100
1/1 - 1s - loss: 5.1966 - accuracy: 0.3265 - 751ms/epoch - 751ms/step
Epoch 9/100
1/1 - 1s - loss: 4.7755 - accuracy: 0.3061 - 754ms/epoch - 754ms/step
Epoch 10/100
1/1 - 1s - loss: 3.4204 - accuracy: 0.3061 - 785ms/epoch - 785ms/step
Epoch 11/100
1/1 - 1s - loss: 2.7867 - accuracy: 0.4490 - 751ms/epoch - 751ms/step
Epoch 12/100
1/1 - 1s - loss: 2.3276 - accuracy: 0.4490 - 748ms/epoch - 748ms/step
Epoch 13

Epoch 95/100
1/1 - 1s - loss: 0.4433 - accuracy: 0.8776 - 871ms/epoch - 871ms/step
Epoch 96/100
1/1 - 1s - loss: 0.4169 - accuracy: 0.8776 - 894ms/epoch - 894ms/step
Epoch 97/100
1/1 - 1s - loss: 0.3618 - accuracy: 0.8776 - 826ms/epoch - 826ms/step
Epoch 98/100
1/1 - 1s - loss: 0.4497 - accuracy: 0.8367 - 819ms/epoch - 819ms/step
Epoch 99/100
1/1 - 1s - loss: 0.6873 - accuracy: 0.8367 - 826ms/epoch - 826ms/step
Epoch 100/100
1/1 - 1s - loss: 0.3408 - accuracy: 0.8571 - 812ms/epoch - 812ms/step


In [10]:
# Get the class indices
class_indices = train_generator.class_indices
#print(f"Class indices: {class_indices}")

# Invert the class indices to get a mapping from index to class label
index_to_class = {v: k for k, v in class_indices.items()}
#print(f"Index to class mapping: {index_to_class}")

In [11]:
# Directory containing the images for testing
test_image_directory = "Test"

# Get list of image files in the directory
image_files = os.listdir(test_image_directory)

# Process and predict for at least 10 images
for idx in range(min(10, len(image_files))):
    img_path = os.path.join(test_image_directory, image_files[idx])
    
    # Load and preprocess the image
    img = load_img(img_path, target_size=(150, 150))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x /= 255.0
    
    # Predict the class of the image
    classes = model.predict(x)
    
    # Print the result
    print(f"{image_files[idx]}: {classes}")
    
    # Convert one-hot encoding to normal class index and label
    predicted_class_index = np.argmax(classes)
    predicted_class_label = index_to_class[predicted_class_index]
    
    # Print the result
    print(f"Predicted class index: {predicted_class_index}")
    print(f"Predicted class label: {predicted_class_label}")
    print(f"======================================================================================================")


1/1 [==============================] - 1s 1s/step
9to6.png: [[3.1082734e-04 1.6891413e-03 2.7065701e-04 2.0004385e-03 1.5140901e-04
  2.9697502e-02 3.4188791e-03 2.3739058e-05 1.2691923e-03 2.1795917e-03
  2.2390570e-06 3.1631058e-03 1.2193162e-03 1.0395057e-02 1.8270976e-04
  2.8175436e-02 1.8943927e-03 1.8746518e-05 1.0957663e-03 9.6913977e-03
  8.4602386e-02 6.5709301e-03 7.2838742e-01 3.9044004e-02 1.4415698e-03
  3.7954139e-04 1.7487258e-02 1.1138067e-03 7.6105942e-05 6.7584793e-04
  2.3371600e-02]]
Predicted class index: 22
Predicted class label: Parking-Everyone-Weekdays-9am To 6pm-Weekend-9am To 6pm
1/1 [==============================] - 0s 45ms/step
A2 Permit.png: [[1.03266336e-06 2.10861504e-06 3.35457067e-08 8.44854912e-06
  2.31522472e-06 1.07735896e-05 2.35925127e-05 5.07091329e-07
  2.53589445e-04 3.74407275e-04 9.12393627e-09 1.03500897e-04
  3.91953741e-04 2.25207659e-05 1.01097816e-04 5.50665645e-05
  4.71730073e-06 4.57748513e-07 3.05808499e-06 2.72890179e-06
  7.7842